In [1]:
from datetime import datetime, timedelta, timezone
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import time

* Defines a function called head that takes two arguments - a file path and an optional argument n (default value 5). 

* The function reads a CSV file from the given path using the csv.reader function.

Refer the code below:

In [2]:
import csv

def head(file_path, n=5):
    with open(file_path) as f:
        reader = csv.reader(f)
        headers = next(reader)
        print('\t'.join(headers))
        for i, row in enumerate(reader):
            if i >= n:
                break
            print('\t'.join(row))

In [3]:
head('HomeC-meter1_2015.csv', n=10)

Date & Time	use [kW]	gen [kW]	House overall [kW]	Dishwasher [kW]	Furnace 1 [kW]	Furnace 2 [kW]	Home office [kW]	Fridge [kW]	Wine cellar [kW]	Garage door [kW]	Kitchen 12 [kW]	Kitchen 14 [kW]	Kitchen 38 [kW]	Barn [kW]	Well [kW]	Microwave [kW]	Living room [kW]	Solar [kW]
01-01-2015 00:00	1.167223333	0	1.167223333	0.000235556	0.229378889	0.331326111	0.01859	0.067467222	0.00442	0.009903889	0.000437222	6.00E-05	7.22E-06	0	0	0	0	0
01-01-2015 00:30	1.171444444	0	1.171444444	0.000225	0.228757778	0.300047778	0.018603889	0.108881111	0.004456111	0.010001111	0.000347222	8.72E-05	7.22E-06	0	0	0	0	0
01-01-2015 01:00	1.151473889	0	1.151473889	0.000229444	0.229445556	0.323098889	0.01862	0.005851111	0.004449444	0.009916111	0.000611667	3.06E-05	6.67E-06	0	0	0	0	0
01-01-2015 01:30	1.398982222	0	1.398982222	0.000208889	0.277065556	0.314398889	0.018593333	0.005925	0.004457222	0.009871667	0.000693333	3.89E-06	7.22E-06	0	0	0	0	0
01-01-2015 02:00	1.080775	0	1.080775	0.000238889	0.228736111	0.30856	0.018637778	

In [4]:
dataset = pd.read_csv('HomeC-meter1_2015.csv')

In [5]:
# Drop the line with missing values
dataset.dropna(inplace=True)

In [6]:
dataset['Date & Time'].head()

0    01-01-2015 00:00
1    01-01-2015 00:30
2    01-01-2015 01:00
3    01-01-2015 01:30
4    01-01-2015 02:00
Name: Date & Time, dtype: object

In [7]:
time_index = pd.date_range('2016-01-01 05:00', periods=len(dataset),  freq='min')  
time_index = pd.DatetimeIndex(time_index)
dataset = dataset.set_index(time_index)
dataset = dataset.drop(['Date & Time'], axis=1)
dataset.iloc[np.r_[0:5,-5:0]].iloc[:,0]

2016-01-01 05:00:00    1.167223
2016-01-01 05:01:00    1.171444
2016-01-01 05:02:00    1.151474
2016-01-01 05:03:00    1.398982
2016-01-01 05:04:00    1.080775
2016-02-01 11:11:00    1.136050
2016-02-01 11:12:00    0.949633
2016-02-01 11:13:00    0.944700
2016-02-01 11:14:00    0.945417
2016-02-01 11:15:00    0.936550
Name: use [kW], dtype: float64

In [8]:
# dataset = dataset.drop(columns=['Date & Time'])
# dataset.shape

In [9]:
dataset.isna().sum()

use [kW]              0
gen [kW]              0
House overall [kW]    0
Dishwasher [kW]       0
Furnace 1 [kW]        0
Furnace 2 [kW]        0
Home office [kW]      0
Fridge [kW]           0
Wine cellar [kW]      0
Garage door [kW]      0
Kitchen 12 [kW]       0
Kitchen 14 [kW]       0
Kitchen 38 [kW]       0
Barn [kW]             0
Well [kW]             0
Microwave [kW]        0
Living room [kW]      0
Solar [kW]            0
dtype: int64

In [10]:
dataset.columns = [col.replace(' [kW]', '') for col in dataset.columns]
dataset.columns

Index(['use', 'gen', 'House overall', 'Dishwasher', 'Furnace 1', 'Furnace 2',
       'Home office', 'Fridge', 'Wine cellar', 'Garage door', 'Kitchen 12',
       'Kitchen 14', 'Kitchen 38', 'Barn', 'Well', 'Microwave', 'Living room',
       'Solar'],
      dtype='object')

In [11]:
data_daily = dataset['House overall'].resample('d').mean()

In [12]:
import numpy as np

class ARIMA:
    def __init__(self, p, d, q):
        self.p = p
        self.d = d
        self.q = q
    
    def fit(self, data):
      self.data = np.array(data).reshape(-1, 1)
      self.n = len(self.data)
      self.mu = np.mean(self.data)
      self.phi = np.zeros((self.p,))
      self.theta = np.zeros((self.q,))
    
    # Initialize residuals
      self.residuals = np.zeros((self.n,))
      self.residuals[:self.p] = self.data[:self.p] - self.mu
    
    # Fit AR(p) model
      if self.p > 0:
          for i in range(self.p, self.n):
              y = self.data[i-self.p:i]
              self.phi = np.linalg.inv(y.T @ y) @ y.T @ self.residuals[i-1]
              self.residuals[i] = self.data[i] - self.mu - self.phi @ self.residuals[i-self.p:i][::-1]
            
    # Fit MA(q) model
      if self.q > 0:
          for i in range(self.q, self.n):
              y = self.residuals[i-self.q:i]
              self.theta = np.linalg.inv(y.T @ y) @ y.T @ self.residuals[i]
              self.residuals[i] -= self.theta @ self.residuals[i-self.q:i][::-1]

                
    def predict(self, n):
        forecast = np.zeros((n,))
        for i in range(n):
            if self.p > 0 and i >= self.p:
                ar = self.residuals[i-self.p:i][::-1] @ self.phi
            else:
                ar = 0
                
            if self.q > 0 and i >= self.q:
                ma = self.residuals[i-self.q:i][::-1] @ self.theta
            else:
                ma = 0
                
            forecast[i] = self.mu + ar + ma
        
        return forecast


The ARIMA class is defined, which takes three parameters: p (the order of the AR component), d (the degree of differencing), and q (the order of the MA component).

* The fit method is defined, which takes in a one-dimensional array of time-series data and pre-processes it for use in the ARIMA model. Specifically, it reshapes the data to a column vector, sets the length of the data, calculates the mean of the data, and initializes the model parameters (phi and theta) and residuals.

* The AR(p) and MA(q) models are fit using two for-loops. For the AR(p) model, the loop iterates over the length of the data and computes the phi parameter using least squares regression. For the MA(q) model, the loop iterates over the length of the data and computes the theta parameter using least squares regression.

* The predict method is defined, which takes in the number of time-steps to forecast and returns an array of forecasted values. The method uses the AR(p) and MA(q) parameters to compute the forecasted values, given the current residuals and mean.

On a broader level, the code implements a basic ARIMA model, which is a popular time-series forecasting method. <br>

The model uses autoregressive (AR) and moving average (MA) components to capture trends and patterns in the data, and can be tuned using the order of these components (p and q) as well as the degree of differencing (d).

In [13]:
size = int(len(data_daily)*0.7)
train = data_daily[:size]
test = data_daily[size:]
print('Number of points in series:', len(data_daily))
print('Number of points in train:', len(train))
print('Number of points in test:', len(test))

Number of points in series: 32
Number of points in train: 22
Number of points in test: 10


* len(data_daily) returns the length of the time series data stored in the data_daily variable.
* size = **int(len(data_daily)*0.7)** calculates 70% of the length of the time series data and stores it in the size variable.
* train = **data_daily[:size]** extracts the first 70% of the time series data and stores it in the train variable.
* test = data_daily[size:] extracts the remaining 30% of the time series data and stores it in the test variable.

print('Number of points in series:', len(data_daily)) prints the length of the entire time series.
print('Number of points in train:', len(train)) prints the length of the training set.
print('Number of points in test:', len(test)) prints the length of the test set.

On a broader level, this code is splitting the time series data into a training set and a test set, with the training set consisting of the first 70% of the data and the test set consisting of the remaining 30%. The code also prints out the number of data points in each set.

In [14]:
arima = ARIMA(p=1, d=0, q=1)

In [16]:
# arima.fit(data_daily)

In [17]:
forecasts = arima.predict(n=3)
print(forecasts)

[0.93039385 0.93039385 0.93039385]


* The predict method of the arima object which was previously defined as an instance of the ARIMA class.
* The predict method takes one argument n which specifies the number of future time steps to predict.

* In this case, n=3, so the method will predict the values for the next 3 time steps.
The predicted values are then stored in the forecasts variable.

Finally, the predicted values are printed using the print() function.